In [ ]:
import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory
import numpy as np

In [ ]:
#不同字符的数量，只有数字所以为10
CHAR_SET_LEN = 10
#图片宽度高度
IMAGE_HEIGHT, IMAGE_WIDTH = 60, 160
#批次大小
BATCH_SIZE = 25
#TFrecord 文件存放路径
TFRECORD_FILE = '/Users/yangming/Downloads/jupyter_notebook/captcha_code/captcha/train.tfrecords'

#placeholder
#x为图片数据文件，y为label，因为是多任务，所以用了4个label
#alexnet 要求传入的大小为224*224
x = tf.placeholder(tf.float32, [None, 224, 224])
y0 = tf.placeholder(tf.float32, [None])
y1 = tf.placeholder(tf.float32, [None])
y2 = tf.placeholder(tf.float32, [None])
y3 = tf.placeholder(tf.float32, [None])

#定义学习率
lr = tf.Variable(0.003, dtype=tf.float32)

#从tfrecord读取数据
#传入的filename实际上就是TFrecord 文件存放路径
def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    #返回文件名和文件
    _, serialized_example = reader.read(filename_queue)
    features = tf.paser_single_example(serialized_example,
                                                       features={
                                                           'image' : tf.FixedLenFeature([],tf.string),
                                                           'label0' : tf.FixedLenFeature([],tf.int64),
                                                           'label1' : tf.FixedLenFeature([],tf.int64),
                                                           'label2' : tf.FixedLenFeature([],tf.int64),
                                                           'label3' : tf.FixedLenFeature([],tf.int64),
                                                       })
    #获取图片数据
    image = tf.decode_raw(features['image'], tf.uint8)
    #tf.train.shuffle_batch必须确定shape
    image = tf.reshape(image, [224, 224])
    #图片预处理
    #cast为类型转换，不过为啥要除以255？
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    #获取label
    label0 = tf.cast(features['label0'], tf.int32)
    label1 = tf.cast(features['label1'], tf.int32)
    label2 = tf.cast(features['label2'], tf.int32)
    label3 = tf.cast(features['label3'], tf.int32)
    
    return image, label0, label1, label2, label3
    